In [16]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

import time
import base64

from captcha_ocr import Captcha_OCR

In [17]:
account = ""
password = ""

In [25]:
def save_captch_img():
    """
    Get and save the current captcha image.
    """

    js = """
    var canvas = document.createElement('canvas');
    var context = canvas.getContext('2d');
    var img = document.getElementById("ctl00_Login1_Image1");

    canvas.height = img.naturalHeight;
    canvas.width = img.naturalWidth;
    context.drawImage(img, 0, 0);

    callback = arguments[arguments.length - 1];
    callback(canvas.toDataURL());
    """
    captcha = chrome.execute_async_script(js).split(",")[1]
    with open("catch/captcha.jpg", 'wb+') as img:
        img.write(base64.b64decode(captcha))

def recognize_captcha():
    ocr.process_image("catch", 50, 22)
    return ocr.recognize()

def login():
    while "service" not in chrome.current_url:
        save_captch_img()
        
        input_account = chrome.find_element_by_id("ctl00_Login1_UserName")
        input_password = chrome.find_element_by_id("ctl00_Login1_Password")
        input_captcha = chrome.find_element_by_id("ctl00_Login1_vcode")
        btn_login = chrome.find_element_by_id("ctl00_Login1_LoginButton")

        input_account.clear()
        input_account.send_keys(account)
        input_password.send_keys(password)
        input_captcha.clear()
        input_captcha.send_keys(recognize_captcha())
        time.sleep(0.5)

        btn_login.click()
        time.sleep(1)

def get_btn_id():
    course = []
    i = 2
    while True:
        btn_id = f"ctl00_MainContent_TabContainer1_tabSelected_gvWishList_ctl0{i}_btnAdd"
        try:
            chrome.find_element_by_id(btn_id)
            course.append(btn_id)
        except:
            break
        i += 2
    return course

def register(btn_id):
    """
    Register the course by id.
    """
    
    btn = chrome.find_element_by_id(btn_id)
    btn.click()

In [21]:
url = "https://course.fcu.edu.tw/"
options = Options()
options.add_argument("--disable-notifications")
chrome = webdriver.Chrome(chrome_options=options)
chrome.get(url)

ocr = Captcha_OCR(list("0123456789"), 4)
ocr.load_model("fcu_course_ocr_v1")

<ipython-input-21-0bb2c2765bb4>:4: DeprecationWarning: use options instead of chrome_options
  chrome = webdriver.Chrome(chrome_options=options)


In [22]:
# login
login()
time.sleep(1)
chrome.find_element_by_id("ctl00_MainContent_TabContainer1_tabSelected_Label3").click()
btn_id = get_btn_id()

In [ ]:
while True:
    for i in btn_id:
        try:
            register(i)
        except:
            btn_id.remove(i)
            break
        time.sleep(3)
    if len(btn_id) == 0:
        break